In [1]:
import keras
import pandas as pd
import numpy as np


Using TensorFlow backend.


In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c54f2e7f40e0418cb540ed70e11f9722 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Diz2ljHGjJ0qPCh-8S3hXOzBdW96RAZdQiYFshSOdsTc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_c54f2e7f40e0418cb540ed70e11f9722.get_object(Bucket='diabetes-donotdelete-pr-n8akslip9njwpr',Key='diabetes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:

columns = ['Glucose','BloodPressure','SkinThickness', 'Insulin', 'BMI']
for column in columns:
    df[column].replace(0, np.NaN, inplace=True)
df.dropna(inplace=True)
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
dataset = df.values
X = dataset[:, 0:8]
Y = dataset[:, 8].astype(int)

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler().fit(X)

X = sc.transform(X)
data = pd.DataFrame (X)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [6]:
#Neural Network
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import  KerasClassifier
from keras.optimizers import Adam

In [7]:
np.random.seed(6)

def model_create():
    model = Sequential()
    
    model.add(Dense(8, input_dim = 8, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, activation= 'sigmoid'))
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = KerasClassifier(build_fn = model_create, verbose = 1)

batch_size = [10,20,40]
epochs = [10, 50, 100]

#Make a dictionary for GridSearch
grid = dict(batch_size = batch_size, epochs = epochs)


#Hyper Parameter optimisation
gridcv = GridSearchCV(estimator = model, param_grid = grid, cv = KFold(random_state = 6), verbose = 10)
resultgrid = gridcv.fit(X, Y)

print("Best: {0}, using {1}".format(resultgrid.best_score_, resultgrid.best_params_))
means  = resultgrid.cv_results_['mean_test_score']
std = resultgrid.cv_results_['std_test_score']
params = resultgrid.cv_results_['params']
for mean, stdev, param in zip(means,std,params):
    print('{0} {{1}} with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
261/261 [==============================] - 1s 4ms/step - loss: 0.6276 - acc: 0.6897
Epoch 2/10
261/261 [==============================] - 0s 504us/step - loss: 0.4803 - acc: 0.6973
Epoch 3/10
261/261 [==============================] - 0s 472us/step - loss: 0.4383 - acc: 0.6973
Epoch 4/10
261/261 [==============================] - 0s 457us/step - loss: 0.4254 - acc: 0.7011
Epoch 5/10
261/261 [==============================] - 0s 519us/step - loss: 0.4130 - acc: 0.8123
Epoch 6/10
261/261 [==============================] - 0s 493us/step - loss: 0.3994 - acc: 0.8276
Epoch 7/10
261/261 [==============================] - 0s 513us/step - loss: 0.3938 - acc: 0.8276
Epoch 8/10
261/261 [==============================] - 0s 503us/step - loss: 0.3910 - acc: 0.8238
Epoch 9/10
261/261 [==============================] - 0s 500us/step - loss: 0.3893 - acc: 0.8391
Epoch 10/10
261/261 [==============================] - 0s 203us/step
[CV]  batch_size=10, epochs=10, score=0.732824427480916, tot

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 1s 3ms/step - loss: 0.6397 - acc: 0.6437
Epoch 2/10
261/261 [==============================] - 0s 547us/step - loss: 0.5225 - acc: 0.6552
Epoch 3/10
261/261 [==============================] - 0s 483us/step - loss: 0.4919 - acc: 0.6552
Epoch 4/10
261/261 [==============================] - 0s 474us/step - loss: 0.4813 - acc: 0.7011
Epoch 5/10
261/261 [==============================] - 0s 464us/step - loss: 0.4743 - acc: 0.7893
Epoch 6/10
261/261 [==============================] - 0s 589us/step - loss: 0.4617 - acc: 0.8123
Epoch 7/10
261/261 [==============================] - 0s 505us/step - loss: 0.4555 - acc: 0.8123
Epoch 8/10
261/261 [==============================] - 0s 555us/step - loss: 0.4483 - acc: 0.7969
Epoch 9/10
261/261 [==============================] - 0s 535us/step - loss: 0.4355 - acc: 0.8123
Epoch 10/10
261/261 [==============================] - 0s 183us/step
[CV]  batch_size=10, epochs=10, score=0.7633587722559921, to

...

261/261 [==============================] - 0s 148us/step - loss: 0.3799 - acc: 0.8391
Epoch 29/100
261/261 [==============================] - 0s 149us/step - loss: 0.3746 - acc: 0.8391
Epoch 30/100
261/261 [==============================] - 0s 160us/step - loss: 0.3757 - acc: 0.8391
Epoch 31/100
261/261 [==============================] - 0s 150us/step - loss: 0.3726 - acc: 0.8467
Epoch 32/100
261/261 [==============================] - 0s 138us/step - loss: 0.3721 - acc: 0.8467
Epoch 33/100
261/261 [==============================] - 0s 178us/step - loss: 0.3712 - acc: 0.8467
Epoch 34/100
261/261 [==============================] - 0s 126us/step - loss: 0.3700 - acc: 0.8506
Epoch 35/100
261/261 [==============================] - 0s 143us/step - loss: 0.3667 - acc: 0.8467
Epoch 36/100
261/261 [==============================] - 0s 169us/step - loss: 0.3668 - acc: 0.8506
Epoch 37/100
261/261 [==============================] - 0s 136us/step - loss: 0.3675 - acc: 0.8467
Epoch 38/100
261/261 [=

262/262 [==============================] - 0s 124us/step - loss: 0.4622 - acc: 0.7634
Epoch 9/100
262/262 [==============================] - 0s 108us/step - loss: 0.4624 - acc: 0.7519
Epoch 10/100
262/262 [==============================] - 0s 137us/step - loss: 0.4578 - acc: 0.7672
Epoch 11/100
262/262 [==============================] - 0s 150us/step - loss: 0.4523 - acc: 0.7710
Epoch 12/100
262/262 [==============================] - 0s 101us/step - loss: 0.4550 - acc: 0.7710
Epoch 13/100
262/262 [==============================] - 0s 196us/step - loss: 0.4512 - acc: 0.7710
Epoch 14/100
262/262 [==============================] - 0s 127us/step - loss: 0.4491 - acc: 0.7748
Epoch 15/100
262/262 [==============================] - 0s 110us/step - loss: 0.4478 - acc: 0.7672
Epoch 16/100
262/262 [==============================] - 0s 135us/step - loss: 0.4451 - acc: 0.7710
Epoch 17/100
262/262 [==============================] - 0s 145us/step - loss: 0.4431 - acc: 0.7824
Epoch 18/100
262/262 [==

262/262 [==============================] - 0s 137us/step - loss: 0.3190 - acc: 0.8664
Epoch 91/100
262/262 [==============================] - 0s 137us/step - loss: 0.3182 - acc: 0.8664
Epoch 92/100
262/262 [==============================] - 0s 140us/step - loss: 0.3210 - acc: 0.8702
Epoch 93/100
262/262 [==============================] - 0s 136us/step - loss: 0.3170 - acc: 0.8740
Epoch 94/100
262/262 [==============================] - 0s 120us/step - loss: 0.3183 - acc: 0.8664
Epoch 95/100
262/262 [==============================] - 0s 180us/step - loss: 0.3186 - acc: 0.8740
Epoch 96/100
262/262 [==============================] - 0s 142us/step - loss: 0.3120 - acc: 0.8702
Epoch 97/100
262/262 [==============================] - 0s 151us/step - loss: 0.3178 - acc: 0.8626
Epoch 98/100
262/262 [==============================] - 0s 116us/step - loss: 0.3116 - acc: 0.8702
Epoch 99/100
262/262 [==============================] - 0s 185us/step - loss: 0.3209 - acc: 0.8740
Epoch 100/100
262/262 [

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.2min finished
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/50
392/392 [==============================] - 2s 6ms/step - loss: 0.6591 - acc: 0.6990
Epoch 2/50
392/392 [==============================] - 0s 249us/step - loss: 0.5070 - acc: 0.7806
Epoch 3/50
392/392 [==============================] - 0s 264us/step - loss: 0.4542 - acc: 0.7755
Epoch 4/50
392/392 [==============================] - 0s 281us/step - loss: 0.4445 - acc: 0.7755
Epoch 5/50
392/392 [==============================] - 0s 256us/step - loss: 0.4363 - acc: 0.7806
Epoch 6/50
392/392 [==============================] - 0s 243us/step - loss: 0.4346 - acc: 0.7832
Epoch 7/50
392/392 [==============================] - 0s 279us/step - loss: 0.4377 - acc: 0.7781
Epoch 8/50
392/392 [==============================] - 0s 252us/step - loss: 0.4300 - acc: 0.7883
Epoch 9/50
392/392 [==============================] - 0s 253us/step - loss: 0.4302 - acc: 0.7857
Epoch 10/50
392/392 [==============================] - 0s 290us/step - loss: 0.4265 - acc: 0.8138
Epoch 11/50
392/392 [==========

In [9]:
from keras.layers import Dropout

def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 50, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=6), verbose = 10)
grid_results = grid.fit(X, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  dropout_rate=0.0, learn_rate=0.001, score=0.7251908542545698, total=   7.2s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.3s remaining:    0.0s


[CV]  dropout_rate=0.0, learn_rate=0.001, score=0.7709923705071894, total=   7.6s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


...

[CV]  dropout_rate=0.0, learn_rate=0.1, score=0.8692307609778184, total=   8.4s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.2min remaining:    0.0s


[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.7328244429508238, total=  10.0s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.7633587900008864, total=  10.3s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.8153846172186044, total=  10.5s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.7480916039634297, total=  11.0s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.7786259683033893, total=  11.0s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.8461538461538461, total=  11.4s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV]  dropout_rate=0.1, learn_rate=0.1, score=0.740458014357181, total=  11.3s
[CV] d

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  4.8min finished
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7908163315483502, using {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7704081731487293 (0.03680002010618718) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7780612282911126 (0.03846914989495609) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7755102005844213 (0.0660907214432982) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.770408172844624 (0.03405153183331555) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7908163315483502 (0.04092945102141571) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7602040793518631 (0.02803235385767455) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7755102118363186 (0.04319313721977383) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7780612252500593 (0.03231747139918912) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7678571422489322 (0.04085047583840199) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [10]:


def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer=init, activation=activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer=init, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 50, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=6), verbose = 10)
grid_results = grid.fit(X, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  activation=softmax, init=uniform, score=0.7480916039634297, total=  10.8s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.8s remaining:    0.0s


[CV]  activation=softmax, init=uniform, score=0.7938931384159409, total=  11.1s
[CV] activation=softmax, init=uniform ................................


...

[CV]  activation=softmax, init=zero, score=0.699999988079071, total=  13.1s
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.8min remaining:    0.0s


[CV]  activation=relu, init=uniform, score=0.6946564767196888, total=  13.1s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.725190845154624, total=  13.6s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.8538461648500882, total=  15.1s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.7480916039634297, total=  13.9s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.74809161397337, total=  14.3s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.8076923122772803, total=  14.9s
[CV] activation=relu, init=zero ......................................
[CV]  activation=relu, init=zero, score=0.6106870242657553, total=  14.1s
[CV] activation=relu, init=zero ............

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  8.5min finished
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7959183720605714, using {'activation': 'softmax', 'init': 'uniform'}
0.7959183720605714 (0.040033630759901254) with: {'activation': 'softmax', 'init': 'uniform'}
0.7933673464826175 (0.05487667530433318) with: {'activation': 'softmax', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'softmax', 'init': 'zero'}
0.7576530627450164 (0.06889867127736996) with: {'activation': 'relu', 'init': 'uniform'}
0.7678571483310388 (0.02805999823586292) with: {'activation': 'relu', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'relu', 'init': 'zero'}
0.7806122523485398 (0.03533071963603891) with: {'activation': 'tanh', 'init': 'uniform'}
0.7729591862583647 (0.04360806090556078) with: {'activation': 'tanh', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'tanh', 'init': 'zero'}
0.7831632660663858 (0.03367863333658302) with: {'activation': 'linear', 'init': 'uniform'}
0.7882653126607135 (0.03010204

In [11]:


def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer='uniform', activation='softmax'))
    model.add(Dense(neuron2, input_dim = 8, kernel_initializer='uniform', activation='softmax'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 50, batch_size = 20, verbose = 0)

# define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=6), refit= True,verbose = 10)
grid_results = grid.fit(X, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ... neuron1=4, neuron2=2, score=0.7328244247509323, total=  17.6s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.7s remaining:    0.0s


[CV] ... neuron1=4, neuron2=2, score=0.7786259592034435, total=  18.4s
[CV] neuron1=4, neuron2=2 ............................................


...

[CV] ... neuron1=4, neuron2=8, score=0.8384615457974948, total=  20.7s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.8min remaining:    0.0s


[CV] ... neuron1=8, neuron2=2, score=0.7709923727821758, total=  19.1s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ... neuron1=8, neuron2=2, score=0.7633587890908918, total=  19.4s
[CV] neuron1=8, neuron2=2 ............................................
[CV] .... neuron1=8, neuron2=2, score=0.830769236271198, total=  20.2s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.7328244247509323, total=  20.5s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.7786259592034435, total=  21.9s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.8384615366275494, total=  21.1s
[CV] neuron1=8, neuron2=8 ............................................
[CV] ... neuron1=8, neuron2=8, score=0.7328244347608727, total=  21.1s
[CV] neuron1=8, neuron2=8 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  9.3min finished
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7908163330688769, using {'neuron1': 16, 'neuron2': 2}
0.7755102054501066 (0.03375555925475344) with: {'neuron1': 4, 'neuron2': 2}
0.7780612293554812 (0.0443391702258339) with: {'neuron1': 4, 'neuron2': 4}
0.7678571484830915 (0.06225392663791184) with: {'neuron1': 4, 'neuron2': 8}
0.7882653135730295 (0.030102044355415795) with: {'neuron1': 8, 'neuron2': 2}
0.7831632654581752 (0.04321805218023576) with: {'neuron1': 8, 'neuron2': 4}
0.7908163318524555 (0.046288036844907156) with: {'neuron1': 8, 'neuron2': 8}
0.7908163330688769 (0.03370616108169202) with: {'neuron1': 16, 'neuron2': 2}
0.7806122549334351 (0.03134281427755803) with: {'neuron1': 16, 'neuron2': 4}
0.7806122518923818 (0.03587765946671833) with: {'neuron1': 16, 'neuron2': 8}


In [12]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = grid.predict(X)

print('Accuracy Score: ',accuracy_score(Y, y_pred))
print('Classification Report: ',classification_report(Y, y_pred))

print(df.iloc[1])

prediction = grid.predict(X[1].reshape(1, -1))
print(prediction)

Accuracy Score:  0.8826530612244898
Classification Report:                precision    recall  f1-score   support

           0       0.95      0.87      0.91       262
           1       0.77      0.92      0.84       130

   micro avg       0.88      0.88      0.88       392
   macro avg       0.86      0.89      0.87       392
weighted avg       0.89      0.88      0.88       392

Pregnancies                   0.000
Glucose                     137.000
BloodPressure                40.000
SkinThickness                35.000
Insulin                     168.000
BMI                          43.100
DiabetesPedigreeFunction      2.288
Age                          33.000
Outcome                       1.000
Name: 4, dtype: float64
[[1]]
